In [ ]:
import scipy.io as sio
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt
import numpy as np
import mat73

In [ ]:
mat_data = mat73.loadmat('indy_20160627_01.mat')

In [ ]:
spikes_times = []

time = -1

for i in range(96):
    spike_time = mat_data['spikes'][i][0]
    if spike_time is not None:
        temp = np.argmax(spike_time)
        time = max(time, temp)
    spikes_times.append(spike_time)

In [ ]:
import numpy as np

def calculate_direction_speed_acceleration(target_pos, cursor_pos, time_points):
    # 计算位移向量
    displacement_vector = target_pos - cursor_pos
    
    # 计算时间差
    time_diff = np.diff(time_points.flatten())  # 将时间点展平，然后计算差分

    # 计算速度向量
    velocity_vector = np.diff(cursor_pos, axis=0) / time_diff[:, np.newaxis]

    # 计算加速度向量
    acceleration_vector = np.diff(velocity_vector, axis=0) / time_diff[1:, np.newaxis]

    # 计算方向角
    direction_angle = np.arctan2(displacement_vector[:, 1], displacement_vector[:, 0])
    
    angle = np.degrees(direction_angle)
    
    # 将角度调整为在0到360度之间
    direction_angle = (angle + 360) % 360

    return direction_angle, velocity_vector, acceleration_vector


In [ ]:
direction_angle, velocity_vector, acceleration_vector = calculate_direction_speed_acceleration(mat_data['target_pos'], mat_data['cursor_pos'], mat_data['t'])

# 打印结果
print("方向角：", direction_angle[0:5])
print("速度向量：", velocity_vector[0:5])
print("加速度向量：", acceleration_vector[0:5])

In [ ]:
def round_to_nearest(x, interval=0.004):
    return [int(round(num / interval) * 4) for num in x]

flat_data = []
for sublist in spikes_times:
    if sublist is not None:
        if sublist.size == 1:
            flat_data.append(sublist.item())
        else:
            for item in sublist:
                flat_data.append(item)
                
flat_data = round_to_nearest(flat_data)
print(flat_data[0:20])

In [ ]:
(mat_data["t"][-1] - mat_data["t"][0]) / 0.004 # 50 单位 = 0.2s

In [ ]:
# 将映射后的结果按照50为间隔进行处理
interval = 200
mapped_t = np.arange(1288000, 1288000 + 93501 * 4, interval)
count_per_interval = np.histogram(flat_data, bins=1871)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
direction_angle_360 = [int(x) for x in direction_angle]

count_360 = [set() for _ in range(360)]

repeated_values = np.repeat(np.arange(0, 1871, 1), 50)

for i, angle in enumerate(direction_angle_360):
    count_360[angle].add(int(repeated_values[i]))

number_angle = [0 for i in range(360)]

for i in range(360):
    tmp = list(count_360[i])
    
    count = 0
    for j in tmp:
        count = count + count_per_interval[0][j]
        
    number_angle[i] = count

In [ ]:
# 绘制 count_per_interval 的柱状图
plt.bar([i for i in range(-180, 180)], number_angle[int(len(number_angle) / 2):-1] + number_angle[0:int(len(number_angle) / 2) + 1], align='edge')

# 添加标签和标题
plt.xlabel('angle')
plt.ylabel('firing rate')
plt.title('tuning curve')

# 显示图表
plt.show()

In [ ]:
def round_to_half(x):
    return [int(round(num * 2)) / 2 for num in x]

velocity_vector_xy = np.array(velocity_vector)

# 计算每个向量的长度
speeds_xy = np.linalg.norm(velocity_vector_xy, axis=1)

speed_half = round_to_half(speeds_xy / 100)

In [ ]:
print(len(list(set(speed_half))))

In [ ]:
count_v = [set() for _ in range(13)]
count_vs = [0 for _ in range(13)]

repeated_values = np.repeat(np.arange(0, 1871, 1), 50)

for i, speed in enumerate(speed_half):
    count_v[int(speed / 0.5)].add(int(repeated_values[i]))
    count_vs[int(speed / 0.5)] = count_vs[int(speed / 0.5)] + 1

number_v = [0 for i in range(13)]

for i in range(13):
    tmp = list(count_v[i])
    
    count = 0
    for j in tmp:
        count = count + count_per_interval[0][j]
        
    number_v[i] = count

print(number_v)
print(count_vs)

vv = [0 for _ in range(13)]

for i, k in enumerate(number_v):
    vv[i] = int(number_v[i] * 2 / count_vs[i])

In [ ]:
# 绘制 count_per_interval 的柱状图
plt.bar([i / 2 for i in range(13)], vv, align='edge')

# 添加标签和标题
plt.xlabel('speed')
plt.ylabel('firing rate')
plt.title('tuning curve')

# 显示图表
plt.show()